In [1]:
pwd

'D:\\data science\\ineuron\\ML Ops\\Simple_app\\notebooks'

In [2]:
from src.get_data import read_params
import argparse
import mlflow
from mlflow.tracking import MlflowClient
from pprint import pprint
import joblib
import os

os.chdir('D:\\data science\\ineuron\\ML Ops\\Simple_app\\notebooks')

def log_production_model(config_path):
    config = read_params(config_path)
    mlflow_config = config["mlflow_config"]
    model_name = mlflow_config["registered_model_name"]
    remote_server_uri = mlflow_config["remote_server_uri"]
    mlflow.set_tracking_uri(remote_server_uri)
    runs = mlflow.search_runs(experiment_ids=1)
    lowest = runs["metrics.mae"].sort_values(ascending=True)[0]
    lowest_run_id = runs[runs["metrics.mae"] == lowest]["run_id"][0]

    client = MlflowClient()
    for mv in client.search_model_versions(f"name='{model_name}'"):
        mv = dict(mv)

        if mv["run_id"] == lowest_run_id:
            current_version = mv["version"]
            logged_model = mv["source"]
            pprint(mv, indent=4)
            client.transition_model_version_stage(name=model_name, version=current_version, stage="Production")
        else:
            current_version = mv["version"]
            client.transition_model_version_stage(name=model_name, version=current_version, stage="Staging")
log_production_model("../params.yaml")

{   'creation_timestamp': 1617886802426,
    'current_stage': 'None',
    'description': '',
    'last_updated_timestamp': 1617886802426,
    'name': 'ElasticNetWineModel',
    'run_id': '2f95f1a77e7242e5ac7f620ffb4c49ed',
    'run_link': '',
    'source': './artifacts/1/2f95f1a77e7242e5ac7f620ffb4c49ed/artifacts/model',
    'status': 'READY',
    'status_message': '',
    'tags': {},
    'user_id': '',
    'version': '1'}
